In [45]:
from kubernetes import client, config

config.load_kube_config()

# use this insted in the pod, will use env vars
# config.load_incluster_config()

In [5]:
v1 = client.CoreV1Api()

In [9]:
for pod in v1.list_namespaced_pod("telegraf").items:
    print("%s\t%s\t%s" % (pod.metadata.name, pod.status.phase, pod.status.pod_ip))

mychart-6869485dc4-9f4vj	Running	172.17.0.9
mychart-telegraf-5979bc89d4-r7k7c	Running	172.17.0.10


/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [18]:
v1.read_namespaced_config_map("telegrafd", "telegraf")

/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'api_version': 'v1',
 'binary_data': None,
 'data': None,
 'kind': 'ConfigMap',
 'metadata': {'annotations': {'meta.helm.sh/release-name': 'mychart',
                              'meta.helm.sh/release-namespace': 'telegraf'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 11, 18, 9, 32, 10, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {'app.kubernetes.io/instance': 'mychart',
                         'app.kubernetes.io/managed-by': 'Helm',
                         'app.kubernetes.io/name': 'mychart',
                         'app.kubernetes.io/version': '1.16.0',
                         'helm.sh/chart': 'mychart-0.1.0'},
              'managed_fields': None,
              'name': 'telegrafd',
              

In [33]:
v1.patch_namespaced_config_map("telegrafd", "telegraf", {"data": {"xxx3.conf": "abc"}})

/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'api_version': 'v1',
 'binary_data': None,
 'data': {'xxx.conf': 'abc', 'xxx2.conf': 'abc', 'xxx3.conf': 'abc'},
 'kind': 'ConfigMap',
 'metadata': {'annotations': {'meta.helm.sh/release-name': 'mychart',
                              'meta.helm.sh/release-namespace': 'telegraf'},
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 11, 18, 9, 32, 10, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {'app.kubernetes.io/instance': 'mychart',
                         'app.kubernetes.io/managed-by': 'Helm',
                         'app.kubernetes.io/name': 'mychart',
                         'app.kubernetes.io/version': '1.16.0',
                         'helm.sh/chart': 'mychart-0.1.0'},
              'managed_fields': 

In [40]:
def clear_configmap(api: client.CoreV1Api, name: str, namespace: str):
    """ drops all config map items """
    cm = api.read_namespaced_config_map(name, namespace)
    if cm.data:
        api.patch_namespaced_config_map(
            name, namespace, {"data": {k: None for k in cm.data}}
        )


clear_configmap(v1, "telegrafd", "telegraf")

/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [48]:
def get_current_namespace() -> str:
    """ get namespace of executing pod, only works inside a pod """
    return open("/var/run/secrets/kubernetes.io/serviceaccount/namespace").read()

In [83]:
# v1.list_namespaced_pod("telegraf").items[0].metadata.owner_references[0]

In [55]:
appsv1 = client.AppsV1Api()

In [63]:
appsv1.read_namespaced_deployment_scale("mychart", "telegraf")

/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'api_version': 'autoscaling/v1',
 'kind': 'Scale',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 11, 18, 13, 21, 25, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'mychart',
              'namespace': 'telegraf',
              'owner_references': None,
              'resource_version': '101774',
              'self_link': '/apis/apps/v1/namespaces/telegraf/deployments/mychart/scale',
              'uid': 'f544b39f-29a0-11eb-bc9c-525400a3d341'},
 'spec': {'replicas': 1},
 'status': {'replicas': 1,
            'selector': 'app.kubernetes.io/instance=mychart,app.kubernetes.io/name=mychart'}}

In [82]:
appsv1.patch_namespaced_deployment_scale(
    "mychart", "telegraf", {"spec": {"replicas": 1}}
)

/nix/store/976dyy4pdsfpjny0xg1w0d1jrcs3cj1n-python3-3.8.5-env/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.42.193'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'api_version': 'autoscaling/v1',
 'kind': 'Scale',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2020, 11, 18, 13, 21, 25, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'mychart',
              'namespace': 'telegraf',
              'owner_references': None,
              'resource_version': '102851',
              'self_link': '/apis/apps/v1/namespaces/telegraf/deployments/mychart/scale',
              'uid': 'f544b39f-29a0-11eb-bc9c-525400a3d341'},
 'spec': {'replicas': 1},
 'status': {'replicas': 0,
            'selector': 'app.kubernetes.io/instance=mychart,app.kubernetes.io/name=mychart'}}